In [ ]:
import torch
from torch import nn, optim
from neurodiffeq import diff
from neurodiffeq.networks import FCNN
from neurodiffeq.temporal import generator_2dspatial_rectangle, generator_2dspatial_segment, generator_temporal
from neurodiffeq.temporal import FirstOrderInitialCondition, BoundaryCondition
from neurodiffeq.temporal import SingleNetworkApproximator2DSpatialSystem
from neurodiffeq.temporal import MonitorMinimal
from neurodiffeq.temporal import _solve_2dspatial_system
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
RE = 100.0

def lid_driven_cavity(u, v, p, x, y):
    momentum_x = u*diff(u, x) + v*diff(u, y) + diff(p, x) - 1/RE * (diff(u, x, order=2) + diff(u, y, order=2))
    momentum_y = u*diff(v, x) + v*diff(v, y) + diff(p, y) - 1/RE * (diff(v, x, order=2) + diff(v, y, order=2))
    continuity = diff(u, x) + diff(v, y)
    return momentum_x, momentum_y, continuity

In [ ]:
# boundary condition on u
u_bc_le = BoundaryCondition(
    form=lambda u, v, p, x, y: u,
    points_generator=generator_2dspatial_segment(size=32, start=(0.0, 0.0), end=(0.0, 1.0))
)
u_bc_ri = BoundaryCondition(
    form=lambda u, v, p, x, y: u,
    points_generator=generator_2dspatial_segment(size=32, start=(1.0, 0.0), end=(1.0, 1.0))
)
u_bc_up = BoundaryCondition(
    form=lambda u, v, p, x, y: u - 1.0,
    points_generator=generator_2dspatial_segment(size=32, start=(0.0, 1.0), end=(1.0, 1.0))
)
u_bc_lo = BoundaryCondition(
    form=lambda u, v, p, x, y: u,
    points_generator=generator_2dspatial_segment(size=32, start=(0.0, 0.0), end=(1.0, 0.0))
)

# boundary condition on v
v_bc_le = BoundaryCondition(
    form=lambda u, v, p, x, y: v,
    points_generator=generator_2dspatial_segment(size=32, start=(0.0, 0.0), end=(0.0, 1.0))
)
v_bc_ri = BoundaryCondition(
    form=lambda u, v, p, x, y: v,
    points_generator=generator_2dspatial_segment(size=32, start=(1.0, 0.0), end=(1.0, 1.0))
)
v_bc_up = BoundaryCondition(
    form=lambda u, v, p, x, y: v,
    points_generator=generator_2dspatial_segment(size=32, start=(0.0, 1.0), end=(1.0, 1.0))
)
v_bc_lo = BoundaryCondition(
    form=lambda u, v, p, x, y: v,
    points_generator=generator_2dspatial_segment(size=32, start=(0.0, 0.0), end=(1.0, 0.0))
)

# pressure point constraint on p
def p_gen():
    while True:
        yield torch.tensor([0.0]), torch.tensor([0.0])
p_bc = BoundaryCondition(
    form=lambda u, v, p, x, y: p,
    points_generator=p_gen
)

# training set and validation set
train_gen_spatial = generator_2dspatial_rectangle(
    size=(16, 16), x_min=0.0, x_max=1.0, y_min=0.0, y_max=1.0
)
valid_gen_spatial = generator_2dspatial_rectangle(
    size=(16, 16), x_min=0.0, x_max=1.0, y_min=0.0, y_max=1.0, random=False
)
monitor = Monitor2DMinimal(check_every=10)

fcnn = FCNN(n_input_units=2, n_output_units=3, n_hidden_units=32, n_hidden_layers=1, actv=nn.Tanh)
fcnn_approximator = SingleNetworkApproximator2DSpatialSystem(
    single_network=fcnn,
    pde=lid_driven_cavity,
    initial_condition=initial_condition,
    boundary_conditions=[
        u_bc_le, u_bc_ri, u_bc_up, u_bc_lo,
        v_bc_le, v_bc_ri, v_bc_up, v_bc_lo,
        p_bc
    ]
)
adam = optim.Adam(fcnn_approximator.parameters())

In [ ]:
%matplotlib notebook
lid_driven_cavity_solution, _ = _solve_2dspatial_system(
    train_generator_spatial=train_gen_spatial,
    valid_generator_spatial=valid_gen_spatial,
    approximator=fcnn_approximator,
    optimizer=adam,
    batch_size=256,
    max_epochs=1000,
    shuffle=True,
    metrics=metrics,
    monitor=monitor
)
%matplotlib inline